## Final Project

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import sklearn as sk
import os
import sys
from sklearn.preprocessing import LabelBinarizer
from six.moves import cPickle as pickle
from six.moves import range
from timeit import default_timer as timer

# Below code inspired by feature extraction technique detailed by Hatham Fayek here:
#    http://haythamfayek.com/2016/04/21/speech-processing-for-machine-learning.html

    #only needed if tarfile is not already extracted.
    import tarfile
    t = tarfile.open('UrbanSound8K.tar.gz', mode="r:gz")
    t.extractall()

In [2]:
raw_sound = pd.read_csv('UrbanSound8K/metadata/UrbanSound8K.csv')
raw_sound.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [3]:
import soundfile as sf
data, samplerate = sf.read('UrbanSound8K/audio_const_rate/fold1/7383-3-0-0.wav')
print(data, samplerate)

[ 0.          0.          0.         ..., -0.00784302 -0.00761414
 -0.00749207] 44100


In [4]:
fold_list = []
for i in range(1,11):
    fold_name = 'fold' + str(i)
    fold_list.append(fold_name)
print(fold_list)

['fold1', 'fold2', 'fold3', 'fold4', 'fold5', 'fold6', 'fold7', 'fold8', 'fold9', 'fold10']


In [5]:
from python_speech_features import mfcc
from python_speech_features import logfbank
import scipy.io.wavfile as wave
from scipy.fftpack import dct 

In [6]:
from os import listdir, system
from os.path import isfile, join
mfcc_data = []
exception_count = 0
for i in range(10):
    mypath = 'UrbanSound8K/audio/'+ fold_list[i] + '/'
    files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    for f in files:
        try:
            fn = mypath + f
            
            sig, rate = sf.read(fn)
            mfcc_feat = mfcc(sig,rate, winlen=0.0232,winstep=0.0116,nfilt=40,numcep=25)
            sample = logfbank(sig,rate,winlen=0.0232,winstep=0.0116,nfilt=40)
        


        except:
            print(f)
            exception_count += 1
            continue

        l_row = raw_sound.loc[raw_sound['slice_file_name']==f].values.tolist()
        label = l_row[0][-1]
        #print(label)
        fold = i+1
        mfcc_data.append([sample, rate, label, fold])  
print("Exceptions: ", exception_count)

Exceptions:  0


In [7]:
len(mfcc_data)

8732

In [8]:
cols = ['sample', 'rate', 'label', 'fold']
mfcc_pd = pd.DataFrame(data=mfcc_data, columns=cols)

In [9]:
# check if same shape for each sample
count = {}
for x in mfcc_pd.index:
    try:
        count[mfcc_pd.loc[x,'sample'].shape[0]] += 1
    except KeyError:
        count[mfcc_pd.loc[x,'sample'].shape[0]] = 1

        
print(max(count.values())/mfcc_pd.shape[0])

0.8389830508474576


In [27]:
# flatten and ravel samples
row_const = max(count)
sample_pd = pd.DataFrame()
for x in mfcc_pd.index:
    sample_pd = sample_pd.append(pd.Series(np.pad(mfcc_pd.loc[x,'sample'],
                                      ((0, row_const-mfcc_pd.loc[x,'sample'].shape[0]),
                                       (0,0)), 'constant').ravel()),
                                         ignore_index=True)
    
# each sample in the "sample" column is now an 1d array. To model, expand features into multiple columns

numpy.ndarray

In [ ]:
# add label and fold info to data.
sample_pd['label'] = fcc_pd['label']
sample_pd['fold'] = fcc_pd['fold']

In [10]:
#pickle.dump(sample_pd, open('2dzeropadded_data.p', 'wb'))

In [11]:
print('done')

done
